Here, I gonna test the *neighbor*, *neigh_modify*, and *processors* commands of *Lammps*. To this end, I choose the details of these commands in the following way:
1. processors Px Py Pz:

    1.1. Px, Py, and Pz are the # of processors in direction x, y, and z, respectively.
    
    1.2. For 2 and 4 cores, I use _processors 1 1 *_.
    
    1.3. For 8, 16, 32 cores, I use _processors 2 2 *_.
2. neigbor rskin bin:

    2.1 rskin is the extra distance beyond the rcutoff of the potential. I use WCA (purely repulsive Lennard-Jones potential) with $r_{cutoff}=2^{1/6}\sigma$ where the size (diameter) of an LJ bead $\sigma=a_m=1.0$ and $a_m$ is the monomers size size, so $r_{skin}=rskin*\sigma$.
    
    2.2 In my test, $rskin=0.2,0.3,0.4,$ and $0.5$.
 
3. neigh_modify delay every check page one:

    3.1 *delay* can be 0 or a multiple of every. Here, $delay=0,1,2,4,10,20$.
    
    3.2 *every* is set to $1,2,$ or $4$.
    
    3.3 *page* is set to $3000$, or $100000$.
    
    3.4 check is always *yes*.
    
    3.5. *one* is set to $300$, or $2000$.

4. I also test *recenter* off and on.

In [ ]:
#%matplotlib notebook
%matplotlib inline
# Importing necessary packages:
import sys
import os
#import scipy.integrate as integrate
#import scipy.special as special
#from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
#import math
import re
from glob import glob
import pandas as pd

In [ ]:
path = os.getcwd()
print(path)
try:
    os.mkdir('results')
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

In [ ]:
files = glob('../log_D26_no_backup/*.log')

In [ ]:
# if rskin needed
neighfile=open("neighbor_testing_D26.txt", "w")
# neigh_modify delay NUM every NUM check YES/NO page NUM one NUM:
neighfile.write('filename,shortname,recenter,rskin,delay,every,check,page,one,temp_last_ts,total_eng_last_ts,press_last_ts,total_time(s),cores,timestep,atoms,ts_per_sec,')
# Section columns: min time, avg time, max time, %varavg, %total"
# Section rows: Pair, Bond, Neigh, Comm, Output, Modify, Other
neighfile.write('pair_avg(s),pair_pct,bond_avg(s),bond_pct,neigh_avg(s),neigh_pct,comm_avg(s),comm_pct,output_avg(s),output_pct,modify_avg(s),modify_pct,other_avg(s),other_pct,dangerous\n')
neighfile.close()

In [ ]:
# rskin
for file in files:
    if len(file.split('rskin')) == 1:
        words=file.split('cpu')
        filename = words[0]+'rskin0.2cpu'+words[-1]
        filename = filename.split('.log')
        filename = filename[0]
        filename = filename.split('/')[-1]
        shortname = filename.split('page')[0]+'rskin'+filename.split('rskin')[-1]
        rskin = filename.split('rskin')[-1].split('cpu')[0]
        recenter = filename.split('_')[-1]
    else:
        filename = file.split('.log')
        filename = filename[0]
        filename = filename.split('/')[-1]
        shortname = filename.split('page')[0]+'rskin'+filename.split('rskin')[-1]
        rskin = filename.split('rskin')[-1].split('cpu')[0]
        recenter = filename.split('_')[-1]
    #with open("neighbor_testing.txt", "a") as neigh_write:
    #print(filename)
    with open(file,'r') as log,\
    open("neighbor_testing.txt", "a") as neighfile:
    
        neighfile.write(filename)
        neighfile.write(",")
        neighfile.write(shortname)
        neighfile.write(",")
        neighfile.write(recenter)
        neighfile.write(",")
        neighfile.write(rskin)
        neighfile.write(",")
        
        line = log.readline()
        
        # The other of while loop are important
        #neigh_modify delay NUM every NUM check YES/NO page NUM one NUM:
        while not(line.startswith('neigh_modify')):
            line = log.readline()
            
        words = line.split()
        # picking the NUMs and Yes/No from neigh_modify command:
        for i in range(int(len(words)/2)): 
            neighfile.write(words[2*i+2])
            neighfile.write(",")
            
        if 'rskin' in file:    
            while not(line.startswith("  100000")):
                line = log.readline()
            words = line.split() # tep Temp E_pair E_mol TotEng Press
            neighfile.write(words[1])#temp_last_ts
            neighfile.write(",")
            neighfile.write(words[4])# total_energy_last_ts
            neighfile.write(",")
            neighfile.write(words[5])# press_last_ts
            neighfile.write(",")
            
        else:
            while not(line.startswith("   10000")):
                line = log.readline()
            words = line.split() # tep Temp E_pair E_mol TotEng Press
            neighfile.write(words[1])#temp_last_ts
            neighfile.write(",")
            neighfile.write(words[4])# total_energy_last_ts
            neighfile.write(",")
            neighfile.write(words[5])# press_last_ts
            neighfile.write(",")
            

        while not(line.startswith('Loop time')):
            line = log.readline()
            
        words = line.split()
        neighfile.write(words[3])#total time
        neighfile.write(",")
        neighfile.write(words[5])# # of cores
        neighfile.write(",")
        neighfile.write(words[8])# total timesteps
        neighfile.write(",")
        neighfile.write(words[11])# total atoms
        neighfile.write(",")
        
        while not(line.startswith('Performance:')):
            line = log.readline()
            
        words = line.split()
        neighfile.write(words[3])# timesteps per second
        neighfile.write(",")
       

        while not(line.startswith('Section')):
            line = log.readline()
        _ = log.readline()
        for i in range(6): # Section rows: Pair, Bond, Neigh, Comm, Output, Modify, Other
            # Section columns: min time, avg time, max time, %varavg, %total"
            line = log.readline()
            sect_min = line.split('|')[2].strip()
            neighfile.write(sect_min)
            neighfile.write(",")
            
            sect_pct = line.split()[-1] # Pair pct of total time
            neighfile.write(sect_pct)
            neighfile.write(",")
        
        line = log.readline()
        sect_min = line.split('|')[2].strip()
        neighfile.write(sect_min)
        neighfile.write(",")
        sect_pct = line.split()[-1] # Pair pct of total time
        neighfile.write(sect_pct)
        
        while not(line.startswith('Dangerous')):
            line = log.readline()
        words = line.split()
        neighfile.write(words[-1]) # # number of dangerous builds
        neighfile.write("\n")       

In [ ]:
# if dangerous build needed 
neighfile=open("neighbor_testing_D26.txt", "w")
# neigh_modify delay NUM every NUM check YES/NO page NUM one NUM:
neighfile.write('filename,ens,run_seg,total_time_s,cores,timestep,atoms,ts_per_sec,')
# Section columns: min time, avg time, max time, %varavg, %total"
# Section rows: Pair, Bond, Neigh, Comm, Output, Modify, Other
neighfile.write('pair_avg_s,pair_pct,bond_avg_s,bond_pct,neigh_avg_s,neigh_pct,comm_avg_s,comm_pct,output_avg_s,output_pct,modify_avg_s,modify_pct,other_avg_s,other_pct,dangerous\n')
neighfile.close()

In [ ]:
# dangerous lists
for file in files:
    filename = file.split('.log')
    filename = filename[0]
    filename = filename.split('/')[-1]
    ens = filename.split('ens')[-1]
    with open(file,'r') as log,\
    open("neighbor_testing_D26.txt", "a") as neighfile:    
        line = log.readline()
        
        # The other of while loop are important
        #neigh_modify delay NUM every NUM check YES/NO page NUM one NUM: 
        j = 1
        while line: 
            if line.startswith('Loop time'):
                neighfile.write(filename)
                neighfile.write(",")
                neighfile.write(ens)
                neighfile.write(",")
                neighfile.write(str(j))#total time
                neighfile.write(",")
                j += 1
                words = line.split()
                neighfile.write(words[3])#total time
                neighfile.write(",")
                neighfile.write(words[5])# # of cores
                neighfile.write(",")
                neighfile.write(words[8])# total timesteps
                neighfile.write(",")
                neighfile.write(words[11])# total atoms
                neighfile.write(",")

            if line.startswith('Performance:'):
                words = line.split()
                neighfile.write(words[3])# timesteps per second
                neighfile.write(",")

            if line.startswith('Section'):
                _ = log.readline()
                for i in range(6): # Section rows: Pair, Bond, Neigh, Comm, Output, Modify, Other
                    # Section columns: min time, avg time, max time, %varavg, %total"
                    line = log.readline()
                    sect_min = line.split('|')[2].strip()
                    neighfile.write(sect_min)
                    neighfile.write(",")

                    sect_pct = line.split()[-1] # Pair pct of total time
                    neighfile.write(sect_pct)
                    neighfile.write(",")

                line = log.readline()
                sect_min = line.split('|')[2].strip()
                neighfile.write(sect_min)
                neighfile.write(",")
                sect_pct = line.split()[-1] # Pair pct of total time
                neighfile.write(sect_pct)
                neighfile.write(",")

            if line.startswith('Dangerous'):
                words = line.split()
                neighfile.write(str(int(words[-1]))) # # number of dangerous builds
                neighfile.write("\n") 
            line = log.readline()

In [ ]:
neigh_data = pd.read_csv("neighbor_testing_D26.txt")
neigh_data['sum_pct']=neigh_data['neigh_pct']+neigh_data['comm_pct']
neigh_data['sum_ave_s']=neigh_data['neigh_avg_s']+neigh_data['comm_avg_s']
neigh_data['est_ttotal_s']=5e7/neigh_data['ts_per_sec']
neigh_data['est_ttotal_h'] = neigh_data['est_ttotal_s'] / 3600
cols_sort = ['cores','sum_pct','neigh_pct','comm_pct','ts_per_sec','atoms']
cols_ascending = [True,True,True,True,False,False]
neigh_data.sort_values(cols_sort,inplace=True,ascending=cols_ascending)
neigh_data.reset_index(inplace=True,drop=True)
cols = ['cores','sum_pct','neigh_pct','comm_pct','ts_per_sec','total_time_s','sum_ave_s','est_ttotal_s','dangerous']

In [ ]:
neigh_data.loc[(neigh_data.atoms==42407) &  (neigh_data.ens==2)]['total_time_s'].count()

In [ ]:
neigh_data.loc[neigh_data.atoms==42407].groupby('ens')['total_time_s'].sum()

In [ ]:
unique_atoms = neigh_data.atoms.drop_duplicates().values
runtime_dict = {'natoms':[],'ncores':[],'nens':[],'runtime_count_sampling':[],'runtime_avg_sampling_h':[],'runtime_count':[],'runtime_avg_h':[]}
for atoms in unique_atoms:
    data = neigh_data.loc[(neigh_data.atoms == atoms) & (neigh_data.run_seg > 1)]
    natoms = atoms
    ncores = data.cores.drop_duplicates().values[0]
    nens = 8
    runtime_count = data.groupby('ens')['total_time_s'].count().sum() # shoud be 8*10=80
    runtime_avg_h = data.groupby('ens')['total_time_s'].sum().mean()/3600
    runtime_dict['natoms'].append(natoms)
    runtime_dict['ncores'].append(ncores)
    runtime_dict['nens'].append(nens)
    runtime_dict['runtime_count_sampling'].append(runtime_count)
    runtime_dict['runtime_avg_sampling_h'].append(runtime_avg_h)
    
    data = neigh_data.loc[(neigh_data.atoms == atoms)]
    natoms = atoms
    ncores = data.cores.drop_duplicates().values[0]
    nens = 8
    runtime_count = data.groupby('ens')['total_time_s'].count().sum() # shoud be 8*11=88
    runtime_avg_h = data.groupby('ens')['total_time_s'].sum().mean()/3600
    runtime_dict['runtime_count'].append(runtime_count)
    runtime_dict['runtime_avg_h'].append(runtime_avg_h)
runtime_df = pd.DataFrame.from_dict(runtime_dict)
runtime_df['runtime_avg_equilibration_h'] = runtime_df['runtime_avg_h'] - runtime_df['runtime_avg_sampling_h']
jnum = int(7e7/5e6) # total n# of timestep / # of timestep in one loop
print(jnum)
runtime_df['runtime_avg_sampling_10times_h'] = runtime_df['runtime_avg_sampling_h']*jnum/(runtime_df['runtime_count_sampling']/8)
runtime_df

In [ ]:
runtime_df.groupby('ncores')['runtime_avg_sampling_10times_h'].max()

In [ ]:
condition2 = (neigh_data.run_seg == 11)
condition2 = 
default_cpu2 = neigh_data.loc[(neigh_data.cores == 2)]
default_cpu4 = neigh_data.loc[(neigh_data.cores == 4)]
default_cpu8 = neigh_data.loc[(neigh_data.cores == 8)]
#default_cpu16 = default_page.loc[default_page.cores == 16]
#default_cpu32 = default_page.loc[default_page.cores == 32]

In [ ]:
default_cpu2.ts_per_sec.min()

In [ ]:
data = default_cpu2
facet_grid = sns.relplot(x="run_seg", y="ts_per_sec", hue='atoms',col="ens",col_wrap=4, data=data)
facet_grid.tight_layout()
facet_grid.savefig("facet_plot.pdf")
#g = sns.FacetGrid(attend, col="subject", col_wrap=4, height=2, ylim=(0, 10))
#g.map(sns.pointplot, "solutions", "score", order=[1, 2, 3], color=".3", ci=None)

In [ ]:
cols = ['cores','total_time_s','sum_pct','neigh_pct','comm_pct','ts_per_sec','atoms','dangerous','est_ttotal_s','est_ttotal_h']
cols_sort =  ['ts_per_sec','sum_pct','neigh_pct','comm_pct','sum_ave_s']
cols_ascending = [False,True,True,True,False]
default_cpu2.reset_index(inplace=True,drop=True)
default_cpu2.sort_values(cols_sort,ascending=cols_ascending)[cols]

In [ ]:
# The column with the highest timestep_per_second on a 2 core machine
default_cpu2.iloc[default_cpu2.ts_per_sec.idxmax()]
# The difference in neigh_pct of this column with the column with lowest neigh_pct is less than 5% 
# however its comm_pct is half of that with the lowest neigh_pct.

In [ ]:
# The above analyze is for a system with 100080 atoms run for 100000 timesteps with timestep/sec=47.309
# what about a system with 10000 atoms that run for 7*10^7 timesteps?
# To answer this question we assume all the algorithm are of order O(N) where N is the number of atoms.
data = default_cpu2
col = 'ts_per_sec'
idx = data[col].idxmax()
idx = 7
col = 'ts_per_sec'
tp_per_sec_mean = default_cpu2[col].min()
print(tp_per_sec_mean)
test_natoms = data.loc[idx,'atoms']
test_nsteps = data.loc[idx,'timestep']
test_ttotal = data.loc[idx,'total_time_s']
test_cores = data.loc[idx,'cores']
#print(test_nsteps/test_ttotal)
natoms = 2e3
nsteps = 5e7
ttotal = (nsteps/tp_per_sec_mean) # seconds
ttotal_hr = ttotal/3600 # hours 
print("The estimated simulation time for a system with {} atoms in {} timesteps on {}-core machine is {} seconds or {} hours.".format(natoms,nsteps,test_cores,ttotal,ttotal_hr))

In [ ]:
default_cpu4.reset_index(inplace=True,drop=True)
default_cpu4.sort_values(cols_sort,ascending=cols_ascending)[cols]

In [ ]:
default_cpu2.est_ttotal_h.max()

In [ ]:
# The column with the highest timestep_per_second on a 4-core machine
default_cpu4.iloc[default_cpu4.ts_per_sec.idxmax()]
# The difference in neigh_pct of this column with the column with lowest neigh_pct is less than 5% 
# however its comm_pct is half of that with the lowest neigh_pct.

In [ ]:
default_cpu4.iloc[default_cpu4[col].idxmin()]['atoms']

In [ ]:
# The above analyze is for a system with 100080 atoms run for 100000 timesteps with timestep/sec=47.309
# what about a system with 10000 atoms that run for 7*10^7 timesteps?
# To answer this question we assume all the algorithm are of order O(N) where N is the number of atoms.
data = default_cpu4
col = 'ts_per_sec'
idx = data[col].idxmax()
#idx = 7
col = 'ts_per_sec'
tp_per_sec_mean = default_cpu4[col].min()
test_atoms = default_cpu4.iloc[default_cpu4[col].idxmin()]['atom']
print(tp_per_sec_mean)
print(test_atoms)
test_natoms = data.loc[idx,'atoms']
test_nsteps = data.loc[idx,'timestep']
test_ttotal = data.loc[idx,'total_time_s']
test_cores = data.loc[idx,'cores']
#print(test_nsteps/test_ttotal)
natoms = 45000
nsteps = 5e7
ttotal = (45000/)(nsteps/tp_per_sec_mean) # seconds
ttotal_hr = ttotal/3600 # hours 
print("The estimated simulation time for a system with {} atoms in {} timesteps on {}-core machine is {} seconds or {} hours.".format(natoms,nsteps,test_cores,ttotal,ttotal_hr))

In [ ]:
default_cpu8.reset_index(inplace=True,drop=True)
default_cpu8.sort_values(cols_sort,ascending=cols_ascending)[cols][:10]

In [ ]:

# The column with the highest timestep_per_second on a 2 core machine
default_cpu8.iloc[default_cpu8.ts_per_sec.idxmax()]
# The difference in neigh_pct of this column with the column with lowest neigh_pct is less than 5% 
# however its comm_pct is half of that with the lowest neigh_pct.

In [ ]:
# The above analyze is for a system with 100080 atoms run for 100000 timesteps with timestep/sec=47.309
# what about a system with 10000 atoms that run for 7*10^7 timesteps?
# To answer this question we assume all the algorithm are of order O(N) where N is the number of atoms.
data = default_cpu8
col = 'ts_per_sec'
idx = data[col].idxmax()
#idx = 7
col = 'ts_per_sec'
tp_per_sec_mean = default_cpu8[col].min()
print(tp_per_sec_mean)
test_natoms = data.loc[idx,'atoms']
test_nsteps = data.loc[idx,'timestep']
test_ttotal = data.loc[idx,'total_time_s']
test_cores = data.loc[idx,'cores']
#print(test_nsteps/test_ttotal)
natoms = 45000
nsteps = 5e7
ttotal = (nsteps/tp_per_sec_mean) # seconds
ttotal_hr = ttotal/3600 # hours 
print("The estimated simulation time for a system with {} atoms in {} timesteps on {}-core machine is {} seconds or {} hours.".format(natoms,nsteps,test_cores,ttotal,ttotal_hr))

In [ ]:
default_cpu16.reset_index(inplace=True,drop=True)
default_cpu16.sort_values(cols_sort,ascending=cols_ascending)[cols]

In [ ]:
col1='neigh_avg(s)'
col2='comm_avg(s)'
col3='sum_ave(s)'
col4='ts_per_sec'
default_cpu2_bar = default_cpu2[['shortname',col1,col2,col3,col4]]
default_cpu4_bar = default_cpu4[['shortname',col1,col2,col3,col4]]
default_cpu8_bar = default_cpu8[['shortname',col1,col2,col3,col4]]
default_cpu16_bar = default_cpu16[['shortname',col1,col2,col3,col4]]
default_cpu32_bar = default_cpu32[['shortname',col1,col2,col3,col4]]
default_page_bar = default_page[['shortname',col1,col2,col3,col4]]
default_cpu2_bar.sort_values([col3,col2,col1],inplace=True,ascending=False)
default_cpu4_bar.sort_values([col3,col2,col1],inplace=True,ascending=False)
default_cpu8_bar.sort_values([col3,col2,col1],inplace=True,ascending=False)
default_cpu16_bar.sort_values([col3,col2,col1],inplace=True,ascending=False)
default_cpu32_bar.sort_values([col3,col2,col1],inplace=True,ascending=False)
default_page_bar.sort_values([col3,col2,col1],inplace=True,ascending=False)

default_cpu2_bar = default_cpu2_bar[['shortname',col1,col2]]
default_cpu4_bar = default_cpu4_bar[['shortname',col1,col2]]
default_cpu8_bar = default_cpu8_bar[['shortname',col1,col2]]
default_cpu16_bar = default_cpu16_bar[['shortname',col1,col2]]
default_cpu32_bar = default_cpu32_bar[['shortname',col1,col2]]
default_page_bar = default_page_bar[['shortname',col1,col2]]

default_cpu2_bar.set_index('shortname',drop=True,inplace=True)
default_cpu4_bar.set_index('shortname',drop=True,inplace=True)
default_cpu8_bar.set_index('shortname',drop=True,inplace=True)
default_cpu16_bar.set_index('shortname',drop=True,inplace=True)
default_cpu32_bar.set_index('shortname',drop=True,inplace=True)
default_page_bar.set_index('shortname',drop=True,inplace=True)

In [ ]:
default_cpu2[['filename','ts_per_sec']]

In [ ]:
#fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16,9))
ax = default_cpu2_bar.plot.barh(stacked=True,figsize=(20, 30))
ax.text(1700,30,"For rskin=0.2, # of timesteps = 10000")
name = 'avg_min'
ax.set_xlabel('Time (sec)')
plt.savefig('neighbor_cpu2_'+name+'.pdf',dpi=300,format='pdf',bbox_inches='tight')